(data-quickstart)=
# Data Analysis Quickstart


## Introduction

Here we'll do a whistlestop tour of data analysis in Python using a structure called a *dataframe*. Dataframes do everything a spreadsheet does, and a whole lot more. At their simplest, dataframes are a tabular representation of data with rows and columns. The data in each column can be anything; text, numbers, Python objects such as lists or dictionaries, or even other dataframes!

The ability to extract, clean, and analyse data is one of the core skills any economist needs. Fortunately, the (open source) tools that are available for data analysis have improved enormously in recent years, and working with them can be a delight——even the most badly formatted data can be beaten into shape.

In this chapter, we'll see analysis on a *single* dataframe using the Star Wars' characters dataset as an example. For a more thorough grounding in using data, see the next chapter.

This chapter uses the [**pandas**](https://pandas.pydata.org/) and [**numpy**](https://numpy.org/) packages. If you're running this code, you may need to install these packages. The Anaconda distribution of Python comes with **pandas** and **numpy** installed. If you don't have these installed, you can install them by running either `conda install packagename` or `pip install packagename` on your computer's command line. You can find a brief guide to installing packages on {ref}`code-preliminaries`.

This chapter is hugely indebted to the fantastic [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/), and both the **pandas** [documentation](https://pandas.pydata.org/docs/user_guide/index.html) and amazing [introductory tutorials](https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/index.html).

## Loading data and checking datatypes

First we must import the packages we'll be using in the rest of this chapter.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Hide cell
# Set seed for reproducibility
np.random.seed(10)

In [ ]:
import matplotlib_inline.backend_inline

# Plot settings
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

# Set max rows displayed for readability
pd.set_option("display.max_rows", 6)

Loading data into a dataframe is achieved with commands like `df = pd.read_csv(...)` or `df = pd.read_stata(...)`. Let's load the Star Wars data from the internet:

In [ ]:
df = (pd.read_csv(
    "https://github.com/aeturrell/coding-for-economists/raw/main/data/starwars.csv",
    index_col=0,
    )
    .dropna(subset=["species"])
    )
# Check info about dataframe
df.info()

### Look at the first few rows with `head()`

In [ ]:
df.head()

```{admonition} Exercise
What happens if you pass a number to `head()`, eg `head(10)`?
```

## Filter rows and columns with conditions using `df.loc[condition(s) or row(s), column(s)]`

`.loc` stands for location and allows you to filter (aka subset) a dataframe. `.loc` works like an index, so it always comes with square brackets, eg `df.loc[...]`.

`loc` takes two arguments. The first is a list of the names of the rows that you'd like to select *or* a condition (ie a list of booleans with the same length as the dataframe) that selects certain rows. Remember, you can easily create a series of booleans by checking a column against a condition, for example `df['column1'] == 'black'`.

The second argument consists of a list of column names you'd like to select. In both cases, `:` is shorthand for 'use all rows' or 'use all columns'. If you have *either* condition(s) *or* column(s) (but not both), you can simply write `df[condition(s)]` or `df[column(s)]`.

Here's an example with a condition built up out of two parts and a list of columns:

In [ ]:
df.loc[(df["hair_color"] == "brown") & (df["eye_color"] == "blue"), ["name", "species"]]

```{admonition} Exercise
Using `loc`, filter the dataframe to `mass` greater than 50 for the `name` and `homeworld` columns
```

## Sort rows or columns with `.sort_values()`

Use `sort_values(columns, ascending=False)` for descending order.

In [ ]:
df.sort_values(["height", "mass"])

```{admonition} Exercise
Using `sort_values`, sort the dataframe by the `name` column.
```

## Choose multiple rows or columns using slices

Slices can be passed by name using `.loc[startrow:stoprow:step, startcolumn:stopcolumn:step]` or by position using `.iloc[start:stop:step, start:stop:step]`.

Choosing every 10th row from the second, and the columns between 'name' and 'gender':

In [ ]:
df.loc[2::10, "name":"gender"]

Note that `loc` only works here with numbers for rows because it just so happens that the names of the rows *are* numbers. If the rows had names that were strings, and we wanted to subset rows by their index position, we would have to use `iloc` instead.

Choosing the first 5 rows and the last 2 columns by index position:

In [ ]:
df.iloc[:5, -2:]

```{admonition} Exercise
Using `.iloc`, display the first 6 rows and last 6 columns.
```

## Randomly selecting a sample using `.sample`

`.sample(n)` randomly selects `n` rows, `.sample(frac=0.4)` selects 40% of the data, `replace=True` samples with replacement, and passing `weights=` selects a number or fraction with the probabilities given by the passed weights. (Note that weights passed should have the same length as the dataframe.)

Taking a sample of 5 rows:

In [ ]:
df.sample(5)

```{admonition} Exercise
Use `.sample` to sample 5% of the dataframe.
```

## Rename with `.rename`

You can rename all columns by passing a function, for instance `df.rename(columns=str.lower)` to put all columns in lower case. Alternatively, use a dictionary to say which columns should be mapped to what:

In [ ]:
df.rename(columns={"homeworld": "home_world"})

## Add new columns with `.assign` or assignment

Very often you will want to create new columns based on existing columns.

![](https://pandas.pydata.org/docs/_images/05_newcolumn_1.svg)

There are two ways to do this. Let's see them both with an example where we'd like to create a new column of height in metres, called `"height_m:`.

- The first, and most commonly used, is called *assignment* and involves just entering the new column name within your dataframe and putting it on the left-hand side of an assignment expression that has an operation based on existing dataframe columns on the right-hand side. For example, `df['height_m'] = df['height']/100`.
- The second is to use the `assign` method on a dataframe directly. In this case, the assignment statement appears inside the brackets but you don't need to write as much text because it's clear from the context that, on the left-hand side of the assignment, we're talking about the given dataframe. An example is `df.assign(height_m=df["height"] / 100)`.

Let's see working examples of both of these assignment methods.

First let's use the assignment approach:

In [ ]:
df['height_m'] = df['height']/100
df.head()

And now with the `.assign` function:

In [ ]:
df = df.assign(height_m=df["height"] / 100)
df.head()

This was added to the end; ideally, we'd like it next to the height column, which we can achieve by sorting the columns (`axis=1`) alphabetically:

In [ ]:
(df.assign(height_m=df["height"] / 100).sort_index(axis=1))

To overwrite existing columns just use `height = df['height']/100` with the `assign` method or `df['height'] = df['height']/100` with an assignment expression.

```{admonition} Exercise
Add a new column that gives the ratio of mass to height.
```

## Summarise numerical values with `.describe()`



In [ ]:
df.describe()

## Group variables values with `.groupby()`



In [ ]:
df.groupby("species")[["height", "mass"]].mean()

```{admonition} Exercise
Find the standard deviation (using `std()`) of masses by `homeworld`.
```

## Add transformed columns using `.transform()`

Quite often, it's useful to put a column into a dataframe that is the result of an intermediate groupby and aggregation. For example, subtracting the group mean or normalisation. Transform does this and *returns a transformed column with the same shape as the original dataframe*. Transform preserves the original index. (There are other methods, such as apply, that return a new dataframe with the groupby variables as a new index.)

Below is an example of transform being used to demean a variable according to the mean by species. Note that we are using lambda functions here. Lambda functions are a quick way of writing functions without needing to give them a name, e.g. `lambda x: x+1` defines a function that adds one to x. In the example below, the `x` in the lambda function takes on the role of mass grouped by species.

In [ ]:
df["mass_demean_species"] = df.groupby("species")["mass"].transform(lambda x: x - x.mean())
df.head()

```{admonition} Exercise
Create a `height_demean_homeworld` column that gives the height column with the mean height by homeworld subtracted.
```

## Make quick charts with `.plot.*`

Including scatter, area, bar, box, density, hexbin, histogram, kde, and line.

In [ ]:
df.plot.scatter("mass", "height", alpha=0.5);

In [ ]:
df.plot.box(column="height");

In [ ]:
df["height"].plot.kde(bw_method=0.3);

## Export results and descriptive statistics

You'll often want to export your results to a latex file for inclusion in a paper, presentation, or poster. Let's say we had some descriptive statistics on a dataframe:

In [ ]:
table = df[["mass", "height"]].agg([np.mean, np.std])
table

You can export this to a range of formats, including string, html, xml, markdown, the *clipboard* (so you can paste it), Excel, and more. In your favourite IDE with a Python language server (eg Code, JupyterLab) start typing `table.to` and a list of possible methods beginning `to` should appear, including `to_string()`.

Here is an example of exporting your **pandas** table to a LaTeX table:

In [ ]:
print(table.to_latex(caption="A Table", label="tab:descriptive"))

Writing to the terminal isn't that useful for getting your paper or report done! To export to a file, use `table.to_latex('file.tex', ...)`.

```{admonition} Exercise
Try exporting the table above using the `to_string("table.txt")` method.

If you are running this locally, the file should appear in the directory in which you are running this notebook.

If you are using Google Colab to do these exercises, you can check that the file exported by running `!ls` in a new code cell to see all files in the current notebook directory. To get the contents of the file you created, run `!cat table.txt`.
```

## Summary

This has been a quick tour of what **pandas** can do, and shows the power of this ubiquitous tool, but we've barely seen a fraction of its features. The next chapter will go deeper into how to use **pandas**.